In [107]:
import pandas
import numpy as np
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

np.set_printoptions(suppress=True)

In [7]:
import pandas

#load data
airports=pandas.read_csv("data/airports.csv",encoding = "latin1")
events_US=pandas.read_csv("data/events_US.csv",encoding = "latin1")
fares=pandas.read_csv("data/fares.csv",encoding = "latin1")
flight_traffic=pandas.read_csv("data/flight_traffic.csv",encoding = "latin1")
stock_prices=pandas.read_csv("data/stock_prices.csv",encoding = "latin1")
weather=pandas.read_csv("data/weather.csv",encoding = "latin1")

In [56]:
airline_ids = list(np.unique(fares['airline_id'])) + list(np.unique(flight_traffic['airline_id']))
airline_ids = np.array(airline_ids[1:])
airport_names = np.unique(list(airports['airport_id']) + list(fares['origin_airport']) + list(fares['destination_airport']) + list(flight_traffic['origin_airport']) + list(flight_traffic['destination_airport']))
airline_ids = np.unique(flight_traffic['airline_id']) # redefine, only use flight traffic airlines
print(airline_ids.shape, airport_names.shape)

(12,) (438,)


In [53]:
inv_names_map = {}
for i in range(airport_names.size):
    inv_names_map[airport_names[i]] = i

In [119]:
def cluster_airlines(column):
    X = np.zeros((12, 438))
    for i in range(X.shape[0]):
        mask = flight_traffic['airline_id'] == airline_ids[i]
        curr_data = flight_traffic.loc[mask]
        origins = np.unique(curr_data[column], return_counts=True) # tup (unique, counts)
        for j in range(origins[0].size):
            X[i, inv_names_map[origins[0][j]]] = origins[1][j]

    total_flights_airlines = np.sum(X, axis=1)
    total_flights_airports = np.sum(X, axis=0)
    nonzero_ind = np.nonzero(total_flights_airports)[0]
    X = X[:, nonzero_ind]
    print('new X shape', X.shape)
    X_norm = X / np.expand_dims(total_flights_airlines, axis=1)
    print(airline_ids)

    cluster_range = np.arange(2, 10, 1)
    for num_clusters in cluster_range:
        clusters = {}
        gmm = GaussianMixture(n_components=num_clusters, random_state=0).fit(X_norm)
        predicted = gmm.predict(X_norm)
        for i in range(predicted.size):
            assignment = predicted[i]
            if assignment not in clusters.keys():
                clusters[assignment] = [airline_ids[i]]
            else:
                clusters[assignment].append(airline_ids[i])
        print(num_clusters, clusters)

In [120]:
cluster_airlines('origin_airport')
print('----------')
cluster_airlines('destination_airport')

new X shape (12, 320)
['AA' 'AS' 'B6' 'DL' 'EV' 'F9' 'HA' 'NK' 'OO' 'UA' 'VX' 'WN']
2 {0: ['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'NK', 'OO', 'UA', 'WN'], 1: ['VX']}
3 {0: ['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'NK', 'OO', 'UA', 'WN'], 1: ['VX'], 2: ['HA']}
4 {0: ['AA', 'B6', 'DL', 'EV', 'F9', 'NK', 'OO', 'UA', 'WN'], 1: ['VX'], 2: ['HA'], 3: ['AS']}
5 {0: ['F9'], 1: ['VX'], 2: ['HA'], 3: ['AS'], 4: ['AA', 'B6', 'DL', 'EV', 'NK', 'OO', 'UA', 'WN']}
6 {0: ['F9'], 1: ['VX'], 2: ['HA'], 3: ['AS'], 4: ['AA', 'DL', 'EV', 'NK', 'OO', 'UA', 'WN'], 5: ['B6']}
7 {0: ['F9'], 1: ['VX'], 2: ['HA'], 3: ['AS'], 4: ['AA', 'EV', 'NK', 'OO', 'UA', 'WN'], 5: ['B6'], 6: ['DL']}
8 {0: ['F9'], 1: ['HA'], 2: ['AA'], 3: ['VX'], 4: ['DL', 'EV', 'UA'], 5: ['AS'], 6: ['NK', 'OO', 'WN'], 7: ['B6']}
9 {0: ['F9'], 1: ['HA'], 2: ['AA'], 3: ['VX'], 4: ['EV', 'UA'], 5: ['AS'], 6: ['NK', 'OO', 'WN'], 7: ['B6'], 8: ['DL']}
----------
new X shape (12, 320)
['AA' 'AS' 'B6' 'DL' 'EV' 'F9' 'HA' 'NK' 'OO' 'UA' 'VX' 'WN']
2